## Tax-advantaged Savings Vehicles

In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def post_tax_return(tax_treat, t_oi_0, t_cg_0, t_oi_T, t_cg_T, r, T):
    # Enter tax rates and rate of return in decimal notation
    # tax_treat_labels: "No tax advantage",  "Nondeductible IRA", "Non-Dividend Stock", "Roth IRA/529 Plan", "Deductible IRA/401(k)/403(b)"
    # tax_treat: ['no_advantage','nondeductible_ira','nondividend_stock', 'roth','401k']
    # Assumes taxes are constant from t=0 to t=T-1 and then jump at t=T
    # Assumes constant rate of return
    if tax_treat == 'no_advantage':
        ret = (1+r*(1-t_oi_0))**(T-1)*(1+r*(1-t_oi_T))   
    elif tax_treat == 'nondeductible_ira':
        ret = (1-t_oi_T)* (1+r)**T  + t_oi_T
    elif tax_treat == 'nondividend_stock':
        ret = (1-t_cg_T)* (1+r)**T  + t_cg_T  
    elif tax_treat == 'roth':
        ret = (1+r)**T
    elif tax_treat == '401k':
        if T == 0:
            ret = ((1-t_oi_0)* (1+r)**T) / (1 - t_oi_0)
        else:
            ret = ((1-t_oi_T)* (1+r)**T) / (1 - t_oi_0)
    else:
        print('Tax treatment not defined')
    return ret

In [3]:
# Parameters (falling ordinary rates)
T_OI_0 = 0.35
T_CG_0 = 0.20
T_OI_T = 0.25
T_CG_T = 0.20
RET = 0.10
HORIZON = 30

In [4]:
# Future value of $1 invested in a taxable account
fv = post_tax_return('no_advantage',T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,HORIZON)
print(f'{fv: .2f}')

 6.68


In [5]:
# Future value of $1 invested in a non-deductible IRA
fv = post_tax_return('nondeductible_ira',T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,HORIZON)
print(f'{fv: .2f}')

 13.34


In [6]:
# Future value of $1 invested in non-dividend/interest-paying asset in a taxable account
fv = post_tax_return('nondividend_stock',T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,HORIZON)
print(f'{fv: .2f}')

 14.16


In [7]:
# Future value of $1 invested in a Roth IRA
fv = post_tax_return('roth',T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,HORIZON)
print(f'{fv: .2f}')

 17.45


In [8]:
# Future value of $1 invested in a 401k
fv = post_tax_return('401k',T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,HORIZON)
print(f'{fv: .2f}')

 20.13


In [9]:
# Create dataframe with accumulations each year
cols = ['no_advantage','nondeductible_ira','nondividend_stock','roth','401k']
df = pd.DataFrame(dtype=float,columns=cols,index=np.arange(HORIZON+1))
for t in np.arange(HORIZON+1):
    for c in cols:
        df.loc[t,c] = post_tax_return(c,T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,t)
df.tail(1)

,no_advantage,nondeductible_ira,nondividend_stock,roth,401k
30,6.676473,13.337052,14.159522,17.449402,20.133926


In [10]:
# Plot the data
df = df.stack().reset_index()
df.columns = ['Year','Vehicle','Withdrawal']
fig = px.line(df, x='Year',y='Withdrawal',color='Vehicle',custom_data=['Vehicle'])
fig.layout.xaxis['title'] = 'Year of Withdrawal'
fig.update_yaxes(
    title='After-Tax FV of $1 of After-tax Initial Investment',
    tickformat='$,.0f'
)
string = '%{customdata}<br>$%{y:,.2f}<extra></extra>'
fig.update_traces(hovertemplate=string)
fig.update_layout(hovermode="x unified")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    ))
fig.show()

In [11]:
# Parameters (higher future ordinary rates)
T_OI_0 = 0.25
T_CG_0 = 0.20
T_OI_T = 0.35
T_CG_T = 0.20
RET = 0.10
HORIZON = 30

# Create dataframe with accumulations each year
cols = ['no_advantage','nondeductible_ira','nondividend_stock','roth','401k']
df = pd.DataFrame(dtype=float,columns=cols,index=np.arange(HORIZON+1))
for t in np.arange(HORIZON+1):
    for c in cols:
        df.loc[t,c] = post_tax_return(c,T_OI_0,T_CG_0,T_OI_T,T_CG_T,RET,t)

# Plot the data
df = df.stack().reset_index()
df.columns = ['Year','Vehicle','Withdrawal']
fig = px.line(df, x='Year',y='Withdrawal',color='Vehicle',custom_data=['Vehicle'])
fig.layout.xaxis['title'] = 'Year of Withdrawal'
fig.update_yaxes(
    title='After-Tax FV of $1 of After-tax Initial Investment',
    tickformat='$,.0f'
)
string = '%{customdata}<br>$%{y:,.2f}<extra></extra>'
fig.update_traces(hovertemplate=string)
fig.update_layout(hovermode="x unified")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
    ))
fig.show()
